#Voila

In [2]:
#!pip install torchtext==0.8.1
!pip install fastai --upgrade -q

     |████████████████████████████████| 194kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 31.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [17]:
import fastai
print(fastai.__version__)

2.3.0


In [18]:
from fastai.vision.all import*
import ipywidgets as widgets
import requests

In [19]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/us?export=download"
    session = requests.Session()
    response = session.get(URL, params = {'id':id}, stream = True)
    token = get_confirm_token(response)
    if token:
        params = {'id':id, 'confirm':token}
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
          return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, 'wb') as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

In [24]:
#https://drive.google.com/file/d/1--h3JK3IQjI7ZSx4pLkQn3a-tGhPzoNy/view?usp=sharing
file_id = '1--h3JK3IQjI7ZSx4pLkQn3a-tGhPzoNy'
destination = './sample_data/röntgenbilder_classifier.pkl'
download_file_from_google_drive(file_id, destination)

#Classifier zu Röntgenbilder


*   Hand
*   Fuß
*   Thorax
*   Schulter
*   Knie


In [26]:
#!cp /content/drive/MyDrive/röntgenbilder_images/röntgenbilder_classifier.pkl ./sample_data
path = Path()
#path.ls(file_exts='.pkl')

inference = load_learner('sample_data/röntgenbilder_classifier.pkl')

In [27]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description = 'Classify')
out_p1 = widgets.Output()
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_p1.clear_output()
    with out_p1: display(img.to_thumb(128,128))
    pred, pred_idx, probs = inference.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Wähle ein Röntgenbild!'),
             btn_upload, btn_run, out_p1, lbl_pred])